In [34]:
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from dotenv import load_dotenv


load_dotenv()

True

In [8]:
temp_review = pd.read_csv("C:/Users/pps/Desktop/Restaurant_Review/Review_Analyzer/Data_Analyze/S_hotel_buffet_review_parse.csv")

In [62]:
review = temp_review.head(10)

In [58]:
sentence = temp_review.Review_Text[0]

In [59]:
sentence
# 처음 방문해 보는 더 파크뷰 랜덤이라 하셨는데 창문 자리로 안내해 주셨어요 금요일 저녁이었는데도 설날 전이 긴 했지만 사람 많았어요 확실히 육류보단 해산물이 더 맛있었어 요대게 수율도 좋았습니다 그리고 정말 배부른 와중에도 너무 맛있게 느낀 마카롱들 디저트 넘넘 맛있더라고요 친절함은 말해 뭐 해 너무 좋았습니다

'직원 분들 너무 친절하시고 음식 맛은 대한민국 호텔 뷔페 넘버 원인데 말해 뭐 해입니다 조금씩 일찍 입장시켜주시는 융통성도 좋아요 쪼금 아쉬운 건 의 외로 과일류 구색이 약하다는 거'

In [60]:
model = ChatOpenAI(model_name = "gpt-3.5-turbo")
output_parser = StrOutputParser()

template = """\
# INSTRUCTION
- 당신은 긍/부정 분류기입니다.
- {category}에 대한 평가가 긍정적인지 부정적인지를 분류하세요.
- {category}에 대한 평가가 없는 경우 '-'을 표시하세요.
- 결과를 다음과 같은 딕셔너리 형식으로 출력하세요.

    "{category}": "긍정/부정/-"
    
# SENTENCE: "직원 분들 너무 친절하시고 음식 맛은 대한민국 호텔 뷔페 넘버 원인데 말해 뭐 해입니다 조금씩 일찍 입장시켜주시는 융통성도 좋아요 쪼금 아쉬운 건 의 외로 과일류 구색이 약하다는 거"
"""

prompt = PromptTemplate.from_template(template)
runnable = {"category": RunnablePassthrough()}
# runnable = {"category":}
chain = runnable | prompt | model

# input_data = [
#     {"category":"맛"},
#     {"category":"서비스"},
#     {"category":"가격"}
# ]

chain.batch([
    {"category":"맛"},
    {"category":"서비스"},
    {"category":"가격"}
])

# await chain.abatch([
#     {"category":"맛", "sentence": sentence},
#     {"category":"서비스", "sentence": sentence},
#     {"category":"가격", "sentence": sentence}
# ])
# await chain.abatch([
#     {"category":"맛"},
#     {"category":"서비스"},
#     {"category":"가격"}
# ])


[AIMessage(content="{'category': '맛'}: 긍정", response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 245, 'total_tokens': 256}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b5ad7523-2d9c-4607-9c2d-7d9783ec5e8a-0', usage_metadata={'input_tokens': 245, 'output_tokens': 11, 'total_tokens': 256}),
 AIMessage(content='{"category": "서비스"}: "긍정"', response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 248, 'total_tokens': 262}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4bbbae6f-f40b-4cc7-a68f-0b25e95fb045-0', usage_metadata={'input_tokens': 248, 'output_tokens': 14, 'total_tokens': 262}),
 AIMessage(content='{\'category\': \'가격\'}: "부정"', response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 248, 'total_tokens': 261}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'st

In [63]:
model = ChatOpenAI(model_name = "gpt-3.5-turbo")
output_parser = StrOutputParser()

template1 = """\
# INSTRUCTION
- 당신은 긍/부정 분류기입니다.
- 맛에 대한 평가가 긍정적인지 부정적인지를 분류하세요.
- 맛에 대한 평가가 없는 경우 '-'을 표시하세요.
- 결과를 다음과 같은 딕셔너리 형식으로 출력하세요.

    "맛": "긍정/부정/-"
    
# SENTENCE: {sentence}
"""

template2 = """\
# INSTRUCTION
- 당신은 긍/부정 분류기입니다.
- 서비스에 대한 평가가 긍정적인지 부정적인지를 분류하세요.
- 서비스에 대한 평가가 없는 경우 '-'을 표시하세요.
- 결과를 다음과 같은 딕셔너리 형식으로 출력하세요.

    "서비스": "긍정/부정/-"
    
# SENTENCE: {sentence}
"""

template3 = """\
# INSTRUCTION
- 당신은 긍/부정 분류기입니다.
- 가격에 대한 평가가 긍정적인지 부정적인지를 분류하세요.
- 가격에 대한 평가가 없는 경우 '-'을 표시하세요.
- 결과를 다음과 같은 딕셔너리 형식으로 출력하세요.

    "가격": "긍정/부정/-"
    
# SENTENCE: {sentence}
"""

prompt1 = PromptTemplate.from_template(template1)
prompt2 = PromptTemplate.from_template(template2)
prompt3 = PromptTemplate.from_template(template3)

chain1 = prompt1 | model | output_parser
chain2 = prompt2 | model | output_parser
chain3 = prompt3 | model | output_parser

combined = RunnableParallel(
    taste = chain1,
    service = chain2,
    price = chain3,
)

for i in range(len(review)):
    result = await combined.abatch([{"sentence": temp_review.Review_Text[i]}])
    print(result)



[{'taste': '{\n    "맛": "긍정"\n}', 'service': '{\n    "서비스": "긍정"\n}', 'price': '{\n    "가격": "-"\n}'}]
[{'taste': '{\n    "맛": "긍정"\n}', 'service': '{\n    "서비스": "긍정"\n}', 'price': '{"가격": "긍정"}'}]
[{'taste': '{\n    "맛": "-"\n}', 'service': '{\n    "서비스": "긍정"\n}', 'price': '{\n    "가격": "긍정"\n}'}]
[{'taste': '{\n    "맛": "부정"\n}', 'service': '{\n    "서비스": "긍정"\n}', 'price': '{\n    "가격": "부정"\n}'}]
[{'taste': '{\n    "맛": "긍정"\n}', 'service': '{\n    "서비스": "긍정"\n}', 'price': '{\n    "가격": "-"\n}'}]
[{'taste': '{\n    "맛": "긍정"\n}', 'service': '{\n    "서비스": "긍정"\n}', 'price': '{\n    "가격": "긍정"\n}'}]
[{'taste': '{\n    "맛": "긍정"\n}', 'service': '{\n    "서비스": "긍정"\n}', 'price': '{\n    "가격": "-"\n}'}]
[{'taste': '{\n    "맛": "긍정"\n}', 'service': '{\n    "서비스": "긍정"\n}', 'price': '{\n    "가격": "긍정"\n}'}]
[{'taste': '{\n    "맛": "긍정"\n}', 'service': '{\n    "서비스": "긍정"\n}', 'price': '{\n    "가격": "긍정"\n}'}]
[{'taste': '{\n    "맛": "긍정"\n}', 'service': '{\n    "서비스": "긍정"\n}', 'price

In [53]:
cat = "맛"
add_runnable = RunnablePassthrough(lambda x: "안뇽")
add_runnable.invoke({"input": "다람쥐"})

{'input': '다람쥐'}

In [54]:
cat = "맛"
add_runnable = RunnablePassthrough.assign(taste = lambda x: "안뇽")
add_runnable.invoke({"input": "다람쥐"})

{'input': '다람쥐', 'taste': '안뇽'}

In [65]:
runnable = {"input": RunnablePassthrough()}
add_runnable = RunnablePassthrough.assign(taste = lambda x: "안뇽")

(runnable | add_runnable).invoke("다람쥐")

{'input': '다람쥐', 'taste': '안뇽'}

---

- 연구원님이 알려주신 prompt안에 하나의 변수만 바꾸고 싶을 때 쓰는 방법이다.

In [64]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from operator import itemgetter

template = """\
# INSTRUCTION
- 당신은 {sentence}에서 {category}에 따라 긍/부정을 분류하는 역할입니다.
- {category}에 대한 반응을 [긍정, 부정, -] 중 하나로 분류하세요
- JSON 형식으로 출력하세요
"""

prompt = PromptTemplate.from_template(template)
model = ChatOpenAI(model_name = "gpt-3.5-turbo")
output_parser = JsonOutputParser()
chain = prompt | model | output_parser

def make_runnable(text):
    return RunnablePassthrough.assign(category = lambda _: text)

combined = RunnableParallel(
    taste = make_runnable("맛") | chain,
    service = make_runnable("서비스") | chain,
    price = make_runnable("가격") | chain,
)

review = "직원분들 너무 친절하시고 음식맛은 대한민국 호텔부페 넘버원인데 말해뭐해 입니다 조금씩 일찍 입장 시켜주시는 융통성도 좋아요 쬐끔 아쉬운건 의외로 과일류 구색이 약하다는 거"

combined.invoke({"sentence": review})

{'taste': {'맛반응': '부정'},
 'service': {'서비스반응': '긍정'},
 'price': {'음식맛': '긍정', '서비스': '긍정', '가격': '부정', '분위기': '긍정'}}